# Perform connective field fitting on HCP data

In [1]:
%load_ext autoreload
%autoreload 2

from vicsompy.surface import Subsurface
import numpy as np
from vicsompy.modeling import MssCf
from vicsompy.surface import CiftiHandler
from vicsompy.subject import HcpSubject

Initialise HcpSubject class.

In [2]:
my_sub = HcpSubject(subject='999999',experiment_id='movie')

Define a function for performing connective field modeling. 

In [3]:
def analyse_subject(expt_id,csub,analysis_name='TEST'):
    my_sub=HcpSubject(subject=csub, experiment_id=expt_id) # Initialise subject

    my_sub.prep_data() # Prep data paths
    my_sub.import_data() # Import data
    my_sub.prepare_out_dirs(analysis_name) # Prepare output directories

    data_train=my_sub.brainmodel.decompose_data(my_sub.all_data.T) # Decompose data into surface and subcortical components.
    data_test=my_sub.brainmodel.decompose_data(my_sub.concatenated_test_sequence.T) # Decompose data into surface and subcortical components.

    nm=MssCf(my_sub,analysis_name=analysis_name) # Initialise sensory_dm object
    nm.make_subsurfaces() # Make subsurfaces

    nm.make_dm(np.nan_to_num(data_train[0])) # Make design matrix.
    nm.prep_pipeline(run_durations=my_sub.run_onsets) # Prepare ridge regression pipeline, specifying run durations for cross-validation.
    nm.fit(np.nan_to_num(my_sub.all_data.T)) # Fit model.
    nm.get_params() # Get parameters.
    nm.test_xval(np.nan_to_num(data_test[0]),np.nan_to_num(my_sub.concatenated_test_sequence.T)) # Test model on held-out data.
    
    nm.save_outcomes() # Save outcomes.
    
    nm.load_lookups() # Load lookup tables for each subsurface.
    nm.load_precomputed_betas() # Load precomputed betas.
    nm.reconstruct_profiles() # Reconstruct profiles via dot product of lookup tables and betas.
    nm.splice_lookups() # Splice lookups via dot product of lookup tables and CF profiles
    nm.save_spliced_lookups() # Saveout these spliced lookups.
    
    nm.prepare_frame(with_spliced=True) # Prepare dataframe of outcomes.
    nm.test_null_model(np.nan_to_num(data_test[0]),my_sub.concatenated_test_sequence) # Also test null model.
    nm.save_frame() # Save dataframe of outcomes
    
    return csub

Do the fitting on the movie subjects.

In [ ]:
mov_fits=[analyse_subject('movie',str(sub),analysis_name='TEST') for sub in my_sub.full_data_subjects]

Do the fitting on the resting state subjects.

In [ ]:
rest_fits=[analyse_subject('rs',str(sub),analysis_name='TEST_REST') for sub in my_sub.full_data_subjects]

Also do fitting on the timecourse average subject.

In [ ]:
av_fit=analyse_subject('movie','999999',analysis_name='TEST')

Also do fitting on subject and movie splits.

In [ ]:
split_subjects=['subsplit_1','subsplit_2','movsplit_1','movsplit_2']

split_fits=[analyse_subject('movie',sub,analysis_name='TEST') for sub in split_subjects]